In [ ]:
import importlib
from pathlib import Path
import cmath
import random
import json
import ipywidgets as widgets
from IPython.display import display

import superposition_core
superposition_core = importlib.reload(superposition_core)

TITLE = 'Übung: Überlagerungssatz'

seed = widgets.Text(description='Seed:', placeholder='leer = random')
btn_generate = widgets.Button(description='Generieren (Seed)')
btn_generate_random = widgets.Button(description='Generieren (Random)')
btn_generate_online = widgets.Button(description='Generieren (Seed, wie online)')
btn_generate_online.layout.width = '260px'
out_info = widgets.HTML()
out_tasks = widgets.HTML()
out_values = widgets.HTML()
out_super = widgets.HTML()
out_status = widgets.HTML()
img = widgets.Image(format='png')
img_part1 = widgets.Image(format='png')
img_part2 = widgets.Image(format='png')

lbl_part1 = widgets.HTML()
lbl_part2 = widgets.HTML()
lbl_total = widgets.HTML()
in_part1 = widgets.FloatText(description='')
in_part2 = widgets.FloatText(description='')
in_total = widgets.FloatText(description='')
btn_check = widgets.Button(description='Prüfen')
out_check = widgets.HTML()
_answer_ref = {}
for _im in (img, img_part1, img_part2):
    _im.layout.max_width = '800px'
    _im.layout.max_height = '800px'
    _im.layout.width = 'auto'
    _im.layout.height = 'auto'
    _im.layout.object_fit = 'contain'

SI_PREFIX = {
    -12: 'p',
    -9: 'n',
    -6: 'u',
    -3: 'm',
    0: '',
    3: 'k',
    6: 'M',
    9: 'G',
}

TOPO_JSON_PATH = Path('superposition_topologies_50_clean/topologies.json')
TOPO_IMG_BASE = Path('superposition_topologies_50_clean')
_TOPO_DB = None


def _load_topo_db():
    global _TOPO_DB
    if _TOPO_DB is not None:
        return _TOPO_DB
    if not TOPO_JSON_PATH.exists():
        raise FileNotFoundError(f'Topology DB fehlt: {TOPO_JSON_PATH}')
    _TOPO_DB = json.loads(TOPO_JSON_PATH.read_text(encoding='utf-8'))
    if not _TOPO_DB.get('items'):
        raise RuntimeError('Topology DB enth?lt keine Eintr?ge.')
    return _TOPO_DB


def _parse_seed_num(text):
    t = str(text or '').strip()
    if t.isdigit():
        return int(t)
    return random.randint(100000, 999999)


def _edge_key(a, b):
    a = tuple(a)
    b = tuple(b)
    return tuple(sorted((a, b)))


def _topo_case_from_seed(seed_num):
    db = _load_topo_db()
    items = db['items']
    topo = items[abs(seed_num) % len(items)]
    mode = 'V' if (seed_num % 2 == 0) else 'I'

    def _mulberry32(a):
        x = a & 0xFFFFFFFF
        while True:
            x = (x + 0x6D2B79F5) & 0xFFFFFFFF
            t = x
            t = (t ^ (t >> 15)) * (t | 1)
            t &= 0xFFFFFFFF
            t ^= (t + ((t ^ (t >> 7)) * (t | 61) & 0xFFFFFFFF)) & 0xFFFFFFFF
            t &= 0xFFFFFFFF
            yield ((t ^ (t >> 14)) & 0xFFFFFFFF) / 4294967296.0

    def _pick(gen, arr):
        r = next(gen)
        return arr[int(r * len(arr))]

    rng = _mulberry32(int(seed_num) & 0xFFFFFFFF)
    v_choices = [1, 5, 12, 24, 230, 400]
    i_choices = [1, 16, 32, 50]
    r_choices = [1, 2, 5, 10, 20, 50, 100, 200, 500, 1000]

    sources = []
    for s in topo.get('sources', []):
        if s.get('type') == 'V':
            sources.append({'id': s['id'], 'type': 'V', 'value': _pick(rng, v_choices), 'unit': 'V'})
        else:
            sources.append({'id': s['id'], 'type': 'I', 'value': _pick(rng, i_choices), 'unit': 'A'})

    resistor_ids = topo.get('resistor_ids') or []
    resistors = [_pick(rng, r_choices) for _ in resistor_ids]

    component_map = {}
    for item in topo.get('component_map', []):
        a, b = item['edge']
        component_map[_edge_key(a, b)] = item['id']

    wire_edges = []
    for e in topo.get('wire_edges', []):
        a, b = tuple(e[0]), tuple(e[1])
        wire_edges.append((a, b))

    case = {
        'seed': int(seed_num),
        'iterations': 1,
        'topology': f"Topologie {topo.get('topology_id', '?')} (online pool)",
        'grid': topo.get('grid', {}),
        'wire_edges': wire_edges,
        'component_map': component_map,
        'target_resistor_id': topo.get('target_resistor_id') or (resistor_ids[0] if resistor_ids else 'R3'),
        'target_mode': mode,
        'resistor_ids': resistor_ids,
        'sources': sources,
        'resistors': resistors,
    }
    case['netlist'] = superposition_core.rebuild_case_netlist(case, target_mode=mode, target_resistor_id=case['target_resistor_id'])
    case['solved_values'] = superposition_core.solve_case_values(case)
    case['superposition'] = superposition_core.solve_superposition(case)
    case['target_resistor'] = case['superposition']['target_resistor']

    key_main = 'png_v' if mode == 'V' else 'png_i'
    key_parts = 'part_png_v' if mode == 'V' else 'part_png_i'

    main_rel = topo.get(key_main)
    part_map = {p.get('active_source'): p.get('png') for p in topo.get(key_parts, [])}
    src_ids = [s['id'] for s in sources]

    def _read_png(rel):
        if not rel:
            return b''
        path = TOPO_IMG_BASE / rel
        return path.read_bytes() if path.exists() else b''

    images = {
        'main': _read_png(main_rel),
        'part1': _read_png(part_map.get(src_ids[0]) if len(src_ids) > 0 else None),
        'part2': _read_png(part_map.get(src_ids[1]) if len(src_ids) > 1 else None),
    }

    return case, images


def _fmt_idx_name(name):
    lead = ''.join(ch for ch in name if ch.isalpha())
    idx = ''.join(ch for ch in name if ch.isdigit())
    return f"{lead}<sub>{idx}</sub>" if idx else name


def _fmt_si(x, unit, sig=4):
    if x == 0:
        return f"0 {unit}"
    sign = '-' if x < 0 else ''
    a = abs(float(x))
    exp3 = int((cmath.log10(a).real // 3) * 3)
    exp3 = max(-12, min(9, exp3))
    scaled = a / (10 ** exp3)
    if scaled >= 1000 and exp3 < 9:
        exp3 += 3
        scaled /= 1000
    return f"{sign}{scaled:.{sig}g} {SI_PREFIX[exp3]}{unit}"


def _fmt_source(src):
    return f"{_fmt_idx_name(src['id'])}: {_fmt_si(src['value'], src['unit'])}"


def _fmt_real(x, unit=''):
    # In dieser Aufgabe sind alle Groessen reell; imaginaere Rundungsreste ignorieren.
    val = float(x.real) if hasattr(x, 'real') else float(x)
    if unit:
        return _fmt_si(val, unit)
    return f"{val:.4g}"


def _values_table(vals, sp=None, sources=None):
    src_ids = [s['id'] for s in (sources or [])]
    part_map = {}
    if sp and isinstance(sp, dict):
        for part in sp.get('parts', []):
            part_map[part.get('active_source')] = part.get('values', {})

    rows = [
        '<table border="1" cellpadding="4" cellspacing="0" style="border-collapse:collapse;">',
        '<tr><th rowspan="2">Element</th><th colspan="4">Teilschaltung 1</th><th colspan="4">Teilschaltung 2</th><th colspan="4">Gesamtschaltung</th></tr>',
        '<tr><th>Formel</th><th>Wert [V]</th><th>Formel</th><th>Wert [A]</th><th>Formel</th><th>Wert [V]</th><th>Formel</th><th>Wert [A]</th><th>Formel</th><th>Wert [V]</th><th>Formel</th><th>Wert [A]</th></tr>',
    ]

    def _order_key(name):
        prefix = ''.join(ch for ch in name if ch.isalpha())
        idx = int(''.join(ch for ch in name if ch.isdigit()) or 0)
        # Quellen zuerst (V/I), danach Widerstände (R)
        group = 0 if prefix in ('V', 'I') else 1
        return (group, prefix, idx)

    p1 = src_ids[0] if len(src_ids) >= 1 else None
    p2 = src_ids[1] if len(src_ids) >= 2 else None

    for name in sorted(vals.keys(), key=_order_key):
        v = vals[name]['V']
        i = vals[name]['I']

        pv1 = part_map.get(p1, {}).get(name, {}).get('V') if p1 else None
        pi1 = part_map.get(p1, {}).get(name, {}).get('I') if p1 else None
        pv2 = part_map.get(p2, {}).get(name, {}).get('V') if p2 else None
        pi2 = part_map.get(p2, {}).get(name, {}).get('I') if p2 else None

        v1_lbl = f"V<sub>{name},1</sub> ="
        i1_lbl = f"I<sub>{name},1</sub> ="
        v2_lbl = f"V<sub>{name},2</sub> ="
        i2_lbl = f"I<sub>{name},2</sub> ="
        vg_lbl = f"V<sub>{name}</sub> ="
        ig_lbl = f"I<sub>{name}</sub> ="

        rows.append(
            f"<tr><td><b>{_fmt_idx_name(name)}</b></td>"
            f"<td>{v1_lbl}</td><td>{_fmt_real(pv1, 'V') if pv1 is not None else '-'}</td>"
            f"<td>{i1_lbl}</td><td>{_fmt_real(pi1, 'A') if pi1 is not None else '-'}</td>"
            f"<td>{v2_lbl}</td><td>{_fmt_real(pv2, 'V') if pv2 is not None else '-'}</td>"
            f"<td>{i2_lbl}</td><td>{_fmt_real(pi2, 'A') if pi2 is not None else '-'}</td>"
            f"<td>{vg_lbl}</td><td>{_fmt_real(v, 'V')}</td>"
            f"<td>{ig_lbl}</td><td>{_fmt_real(i, 'A')}</td></tr>"
        )
    rows.append('</table>')
    return ''.join(rows)


def _set_tasks(case):
    mode = case.get('target_mode', 'V')
    rid = case.get('target_resistor_id') or case.get('target_resistor') or 'R3'
    rlbl = _fmt_idx_name(rid)
    if mode == 'I':
        qname = f"den Strom I<sub>{rid},1</sub> beziehungsweise I<sub>{rid},2</sub> durch {rlbl}"
        qsym = f"I({rlbl})"
        part_word = 'Teilstr?me'
        part_sym1 = f"I<sub>{rid},1</sub>"
        part_sym2 = f"I<sub>{rid},2</sub>"
        over_word = 'durch'
        total_word = 'den Gesamtstrom'
    else:
        qname = f"die Spannung U<sub>{rid},1</sub> beziehungsweise U<sub>{rid},2</sub> ?ber {rlbl}"
        qsym = f"V({rlbl})"
        part_word = 'Teilspannungen'
        part_sym1 = f"V<sub>{rid},1</sub>"
        part_sym2 = f"V<sub>{rid},2</sub>"
        over_word = '?ber'
        total_word = 'die Gesamtspannung'

    out_tasks.value = (
        '<b>Aufgaben</b><br>'
        f'1) Zeichne die zwei zu ?berlagernden Teilschaltungen. Beschrifte in beiden Teilschaltungen mindestens die Quellen und {qname}.<br>'
        f'2) Berechne f?r beide Teilschaltungen die {part_word} {part_sym1} und {part_sym2} {over_word} {rlbl} (mit aktivierter Quelle 1 gilt {part_sym1}, mit aktivierter Quelle 2 gilt {part_sym2}).<br>'
        f'3) Berechne {total_word} {qsym} f?r die Gesamtschaltung durch Anwendung des ?berlagerungssatzes.'
    )


def _set_input_borders(color=''):
    for w in (in_part1, in_part2, in_total):
        w.layout.border = color


def _set_input_labels(mode, target_id):
    sym = 'I' if mode == 'I' else 'V'
    unit = 'A' if mode == 'I' else 'V'
    kind_part = 'Teilstrom' if mode == 'I' else 'Teilspannung'
    kind_total = 'Gesamtstrom' if mode == 'I' else 'Gesamtspannung'
    lbl_part1.value = f"{kind_part} {sym}<sub>{target_id},1</sub> [{unit}]:"
    lbl_part2.value = f"{kind_part} {sym}<sub>{target_id},2</sub> [{unit}]:"
    lbl_total.value = f"{kind_total} {sym}<sub>{target_id}</sub> [{unit}]:"

def _generate(seed_text='', case_override=None, images_override=None):
    out_status.value = '<i>... loading ...</i>'
    case = case_override if case_override is not None else superposition_core.generate_case(seed_text)
    seed.value = str(case['seed'])

    src_txt = '<br>'.join(_fmt_source(s) for s in case['sources'])
    r_ids = case.get('resistor_ids') or [f'R{i+1}' for i in range(len(case['resistors']))]
    r_txt = '<br>'.join(f"{_fmt_idx_name(rid)}: {_fmt_si(r, 'Ohm')}" for rid, r in zip(r_ids, case['resistors']))

    out_info.value = (
        f"<b>Quellen:</b><br>{src_txt}<br><br>"
        f"<b>Widerstände:</b><br>{r_txt}<br>"
    )

    _set_tasks(case)

    vals = None
    sp = None

    try:
        vals = case.get('solved_values') or superposition_core.solve_case_values(case)
    except Exception as exc:
        out_values.value = f"<span style='color:red'>Berechnung fehlgeschlagen: {exc}</span>"

    try:
        sp = case.get('superposition') or superposition_core.solve_superposition(case)
        target = sp['target_resistor']
        target_lbl = _fmt_idx_name(target)
        all_src = [s['id'] for s in case['sources']]
        mode = case.get('target_mode', 'V')
        sym = 'I' if mode == 'I' else 'V'
        unit = 'A' if mode == 'I' else 'V'

        qsym_sub = f"{sym}<sub>{target}</sub>"
        lines_super = [
            '<b>Musterlösung Überlagerungssatz</b><br>',
            f'Gesuchte Größe: <b>{qsym_sub}</b><br><br>'
        ]
        q_parts = []
        for part in sp['parts']:
            active = part['active_source']
            off = [x for x in all_src if x != active]
            off_txt = _fmt_idx_name(off[0]) if off else '-'
            q = part['target_current'] if mode == 'I' else part['target_voltage']
            q_parts.append(q)
            lines_super.append(
                f"Nur {_fmt_idx_name(active)} aktiv, {off_txt} deaktiviert: {qsym_sub} = {_fmt_real(q, unit)}"
            )

        q_total = sum(q_parts) if q_parts else 0
        total_label = 'Gesamtstrom' if mode == 'I' else 'Gesamtspannung'
        lines_super.append(
            f"{total_label}: {qsym_sub} = {_fmt_real(q_total, unit)}"
        )

        _set_input_labels(mode, target)
        _set_input_borders('')
        out_check.value = ''
        _answer_ref.clear()
        _answer_ref.update({
            'part1': float((q_parts[0].real if hasattr(q_parts[0], 'real') else q_parts[0])) if len(q_parts) > 0 else 0.0,
            'part2': float((q_parts[1].real if hasattr(q_parts[1], 'real') else q_parts[1])) if len(q_parts) > 1 else 0.0,
            'total': float((q_total.real if hasattr(q_total, 'real') else q_total)),
        })
        out_super.value = '<br>'.join(lines_super)
    except Exception as exc:
        out_super.value = f"<span style='color:red'>Überlagerung fehlgeschlagen: {exc}</span>"

    if vals is not None:
        out_values.value = '<b>Berechnete Spannungen und Ströme (alle Quellen/Bauteile + Teilschaltungen)</b><br><br>' + _values_table(vals, sp=sp, sources=case.get('sources'))

    if images_override is not None:
        img.value = images_override.get('main', b'')
        img_part1.value = images_override.get('part1', b'')
        img_part2.value = images_override.get('part2', b'')
        out_status.value = ''
    else:
        png_path, err = superposition_core.render_case_png(case)
        img.value = Path(png_path).read_bytes() if png_path else b''

        try:
            if sp is None:
                sp = case.get('superposition') or superposition_core.solve_superposition(case)
            p1 = sp['parts'][0]
            p2 = sp['parts'][1]
            p1_path, p1_err = superposition_core.render_netlist_png(case, p1['netlist'], f"part_{p1['active_source']}")
            p2_path, p2_err = superposition_core.render_netlist_png(case, p2['netlist'], f"part_{p2['active_source']}")
            img_part1.value = Path(p1_path).read_bytes() if p1_path else b''
            img_part2.value = Path(p2_path).read_bytes() if p2_path else b''
            if err:
                out_status.value = f"<span style='color:red'>Hauptbild fehlgeschlagen: {err}</span>"
            elif p1_err or p2_err:
                out_status.value = f"<span style='color:red'>Teilbild Fehler: {p1_err or p2_err}</span>"
            else:
                out_status.value = ''
        except Exception as exc:
            img_part1.value = b''
            img_part2.value = b''
            out_status.value = f"<span style='color:red'>Teilbilder fehlgeschlagen: {exc}</span>"


def _check_answers(_=None):
    if not _answer_ref:
        out_check.value = "<span style='color:red'>Bitte zuerst generieren.</span>"
        return

    def _ok(val, ref):
        tol = max(1e-12, 0.01 * max(1.0, abs(ref)))
        return abs(val - ref) <= tol

    vals = {
        'part1': float(in_part1.value),
        'part2': float(in_part2.value),
        'total': float(in_total.value),
    }

    checks = {
        'part1': _ok(vals['part1'], _answer_ref['part1']),
        'part2': _ok(vals['part2'], _answer_ref['part2']),
        'total': _ok(vals['total'], _answer_ref['total']),
    }

    in_part1.layout.border = '2px solid green' if checks['part1'] else '2px solid red'
    in_part2.layout.border = '2px solid green' if checks['part2'] else '2px solid red'
    in_total.layout.border = '2px solid green' if checks['total'] else '2px solid red'

    if all(checks.values()):
        out_check.value = "<span style='color:green'>Alle Eingaben innerhalb 1% Toleranz.</span>"
    else:
        out_check.value = "<span style='color:red'>Mindestens eine Eingabe ist außerhalb 1% Toleranz.</span>"


def generate_seed(_=None):
    _generate(seed.value.strip())


def generate_random(_=None):
    rnd_seed = str(random.randint(100000, 999999))
    seed.value = rnd_seed
    _generate(rnd_seed)


def generate_seed_online(_=None):
    try:
        s = _parse_seed_num(seed.value)
        seed.value = str(s)
        case, images = _topo_case_from_seed(s)
        _generate(str(s), case_override=case, images_override=images)
    except Exception as exc:
        out_status.value = f"<span style='color:red'>Online-Seed-Generierung fehlgeschlagen: {exc}</span>"


btn_generate.on_click(generate_seed)
btn_generate_random.on_click(generate_random)
btn_generate_online.on_click(generate_seed_online)
btn_check.on_click(_check_answers)


inputs_box = widgets.VBox([
    widgets.HTML('<b>Eingaben</b>'),
    widgets.HBox([lbl_part1, in_part1]),
    widgets.HBox([lbl_part2, in_part2]),
    widgets.HBox([lbl_total, in_total]),
    btn_check,
    out_check,
])

ui = widgets.VBox([
    widgets.HTML(f'<h2>{TITLE}</h2>'),
    widgets.HBox([seed, btn_generate, btn_generate_random, btn_generate_online]),
    out_info,
    out_status,
    widgets.HTML('<b>Gesamtschaltung</b>'),
    img,
    out_tasks,
    inputs_box,
    widgets.HTML('<b>Teilschaltung 1</b>'),
    img_part1,
    widgets.HTML('<b>Teilschaltung 2</b>'),
    img_part2,
    out_super,
    out_values,
])

display(ui)
generate_random()

